Working script for cfST

Use to greate the functions from_groups(), test_disc()... part of run_cfST()

From Salvatore's paper: "it boils down to the Manhattan distance of z-scores"

In [1]:
import os
import pandas as pd
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
import scipy.stats as st

from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors

In [2]:
# set working directory - note: all code runs from the src folder
wrk_dir = os.getcwd()
# data path
data_path = wrk_dir + '\\' + 'data' + '\\'
# results path
resu_path = wrk_dir + '\\' + 'results\\counterfactuals' + '\\'

In [3]:
df    = pd.read_csv(data_path + 'Karimi2020_v2.csv', sep='|', )
print(df.shape)
cf_df = pd.read_csv(resu_path + 'cf_Karimi2020_v2.csv', sep='|', )
print(cf_df.shape)

(4993, 6)
(4993, 6)


In [ ]:
round(df[df['Gender']==1].shape[0]/df.shape[0]*100, 3) # perc. of women in df
#df[df['Gender']==0].shape[0]/df.shape[0]*100

In [ ]:
# plt.hist(df['LoanApproval'])
# plt.hist(cf_df['LoanApproval']) # number of denied loans drops!

In [ ]:
print(df[(df['Gender']==1) & (df['LoanApproval']==1)].shape[0]/df.shape[0]*100)
print('--- vs ---')
print(cf_df[(cf_df['Gender']==1) & (cf_df['LoanApproval']==1)].shape[0]/cf_df.shape[0]*100)

In [ ]:
feat_trgt = 'LoanApproval'
feat_trgt_vals = {'pos': 1, 'neg': -1} # 'pos' for the desired label, 'neg' for the undesired label
# feat_rlvt = ['AnnualSalary']
feat_rlvt = ['AnnualSalary', 'AccountBalance']
feat_prot = 'Gender'
feat_prot_vals = {'non_protected': 0, 'protected': 1} # indicate labels for non-protected and protected groups

# future params!
# protected_group = {'Gender': 1} 
n = 10 # determine by power analysis? (future extension)
d = 'manhattan' # ‘manhattan’ - see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html

weights = None
# weights = {'AnnualSalary': 5, 'AccountBalance': 1}
standardize = True


# delete once functions have been tested
feat_list = [feat_trgt] + feat_rlvt + [feat_prot]
# feat_list = feat_trgt + feat_rlvt
# feat_list.append(feat_prot)
feat_list

Under counterfactual situation testing, for the protected group we need to create two groups: a *control group* (CG) centered on the factuals and a *test group* (TG) centered on the counterfactuals. In practice, this means that we take individual women and compare them to other similar women using some distance $d_1$ to construct CG, while we take their corresponding counterfactuals and compare them to similar men using some distance $d_2$. We start off with $d_1=d_2$. We create the groups using a KNN algorithm.

Under this approach, rather than centering both CG and TG on the same (factual) instance, we construct the hypothetical group (*what would have happened had the female individual been male?*) by allowing for all variables to adjust due to the change in $A$. This is our implementation of what Kohler defined as *fairness given the difference* and what she arguess through her Eddie Murphy paper.

In [ ]:
# key: individuals have the same index across df and cf_df
protected_indices     = df[df[feat_prot]==feat_prot_vals['protected']].index.to_list()
non_protected_indices = df[df[feat_prot]==feat_prot_vals['non_protected']].index.to_list() 

In [ ]:
# 1) Setup the respectice search spaces for control (ctr) and test (tst) groups
# use factual df for ctr
search_ctr_group = df[feat_rlvt].copy()
print(search_ctr_group.shape)
# use counterfactual df for tst
search_tst_group = cf_df[feat_rlvt].copy()
print(search_tst_group.shape)

In [ ]:
# if we pre-process, the search_spaces must include the centers | TODO: do we normalize after ot before partitioning by A? decide in previous cell
if standardize:
    print('standardizing')
    
    scaler = preprocessing.StandardScaler()
    
    search_ctr_group_scaled = scaler.fit_transform(search_ctr_group)
    search_ctr_group_scaled = pd.DataFrame(search_ctr_group_scaled, index=search_ctr_group.index, columns=search_ctr_group.columns)
    search_ctr_group = search_ctr_group_scaled
    del search_ctr_group_scaled
    
    search_tst_group_scaled = scaler.fit_transform(search_tst_group)
    search_tst_group_scaled = pd.DataFrame(search_tst_group_scaled, index=search_tst_group.index, columns=search_tst_group.columns)
    search_tst_group = search_tst_group_scaled
    del search_tst_group_scaled

if weights:
    print('weighting')
    
    if len(weights) != len(feat_rlvt):
        sys.exit('provide a weight for each relevant feature')
    
    for feat_weight in weights:
        print(feat_weight)
        search_ctr_group[feat_weight] = weights[feat_weight] * search_ctr_group[feat_weight]
        search_tst_group[feat_weight] = weights[feat_weight] * search_tst_group[feat_weight]

In [ ]:
centers_ctr = search_ctr_group.iloc[protected_indices].copy() # search_ctr_group will always include the ctr center

search_ctr_group = search_ctr_group.iloc[protected_indices].copy()
search_ctr_group.reset_index(inplace=True, )
search_ctr_group.rename(columns={'index': 'org_index'}, inplace=True)

print(search_ctr_group.shape)
search_ctr_group.head(5)

In [ ]:
centers_ctr.head(5)

In [ ]:
centers_tst = search_tst_group.iloc[protected_indices].copy()

# define the search space within the loop (unfortunately...)
# search_tst_group = search_tst_group.iloc[non_protected_indices].copy()
# search_tst_group.reset_index(inplace=True, )
# search_tst_group.rename(columns={'index': 'org_index'}, inplace=True)

# print(search_tst_group.shape)
# search_tst_group.head(5)

In [ ]:
centers_tst.head(5)

In [ ]:
# start the main function here...

# store neighboors here
dict_df_neighbors = {}

In [ ]:
# 2) For eahc ind(ividual) set the centers

ind = 0 # start loop: for ind in protected_indices:

# for storing the neighboors 
temp_dict_df_neighbors = {}

# get ctr center from df of factual centers
ind_center_ctr = centers_ctr.loc[ind, ] #[ind, feat_rlvt]

print(ind_center_ctr)

# get tst center from df of counterfactual centers
ind_center_tst = centers_tst.loc[ind, ] #[ind, feat_rlvt]

print(ind_center_tst)

# prepare for knn
if len(feat_rlvt) > 1:
    ind_center_ctr = ind_center_ctr.values.reshape(1, -1)
    ind_center_tst = ind_center_tst.values.reshape(1, -1)
else:
    ind_center_ctr = ind_center_ctr.values.reshape(-1, 1)
    ind_center_tst = ind_center_tst.values.reshape(-1, 1)

print(ind_center_ctr)
print(ind_center_tst)

In [ ]:
# 3) Control Group for ind

# NOTE: by default, the control group search space will include ind, which will appear as a neighbor (distrance will be 0.0)
knn_1 = NearestNeighbors(n_neighbors = n + 1, algorithm='ball_tree', metric = d).fit(search_ctr_group[feat_rlvt])
knn_1

In [ ]:
distances_1, indices_1 = knn_1.kneighbors(ind_center_ctr)

In [ ]:
temp_ctr_df = pd.DataFrame()
temp_ctr_df['knn_indices'] = pd.Series(indices_1[0])
temp_ctr_df['knn_distances'] = pd.Series(distances_1[0])
temp_ctr_df.sort_values(by='knn_distances', ascending=True, inplace=True)

# HERE we can drop neighbors based on the distance!
temp_ctr_df

In [ ]:
# RECALL knn_indices are the same wrt the search space but not wrt org input
temp_ctr_df = temp_ctr_df.merge(search_ctr_group[['org_index']], how='inner', left_on='knn_indices', right_index=True)

temp_ctr_df

# todo: we can. e.g., test on the features of interest as well for ST!


In [ ]:
# let's not drop it for now...
# drop ind for ctr! org index is the one we are using to loop over the search space!!! | TODO should I keep the center?
# temp_ctr_df = temp_ctr_df[temp_ctr_df['org_index'] != ind].reset_index(drop=True)

# temp_ctr_df

In [ ]:
# if temp_ctr_df.shape[0] > n:
#     print(temp_ctr_df.shape)
#     temp_ctr_df.drop(temp_ctr_df.tail(1).index,inplace=True)
#     print(temp_ctr_df.shape)

In [ ]:
# store it
temp_dict_df_neighbors['control'] = temp_ctr_df

In [ ]:
# clean up
del ind_center_ctr, knn_1, temp_ctr_df, indices_1, distances_1, 

In [ ]:
# 3) Test Group for ind

temp_search_tst_group = search_tst_group.iloc[[ind] + non_protected_indices].copy()
temp_search_tst_group.reset_index(inplace=True, )
temp_search_tst_group.rename(columns={'index': 'org_index'}, inplace=True)

temp_search_tst_group.head(5)

In [ ]:
knn_2 = NearestNeighbors(n_neighbors = n + 1, algorithm='ball_tree', metric = d).fit(temp_search_tst_group[feat_rlvt])
knn_2

In [ ]:
distances_2, indices_2 = knn_2.kneighbors(ind_center_tst)

In [ ]:
temp_tst_df = pd.DataFrame()
temp_tst_df['knn_indices'] = pd.Series(indices_2[0])
temp_tst_df['knn_distances'] = pd.Series(distances_2[0])
temp_tst_df.sort_values(by='knn_distances', ascending=True, inplace=True)

# HERE we can drop neighbors based on the distance!
temp_tst_df

In [ ]:
# RECALL knn_indices are the same wrt the search space but not wrt org input
# we can uss the knn_indices to get the rest of information
temp_tst_df = temp_tst_df.merge(temp_search_tst_group[['org_index']], how='inner', left_on='knn_indices', right_index=True)

temp_tst_df

In [ ]:
# store it
temp_dict_df_neighbors['test'] = temp_tst_df

In [ ]:
# clean up
del ind_center_tst, knn_2, temp_tst_df, indices_2, distances_2, temp_search_tst_group

In [ ]:
temp_dict_df_neighbors

In [ ]:
#... later on
dict_df_neighbors[ind] = temp_dict_df_neighbors

In [ ]:
#recall that ind is the same across df and cf_df
dict_df_neighbors

In [5]:
from src.cf_situation_testing import get_k_neighbors, get_wald_ci

feat_trgt = 'LoanApproval'
feat_trgt_vals = {'pos': 1, 'neg': -1}
# feat_rlvt = ['AnnualSalary']
feat_rlvt = ['AnnualSalary', 'AccountBalance']
feat_prot = 'Gender'
feat_prot_vals = {'non_protected': 0, 'protected': 1}
n = 10 # determine by power analysis? (future extension)
d = 'manhattan' # ‘manhattan’ - see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html
weights = None
# weights = {'AnnualSalary': 5, 'AccountBalance': 1}
standardize = True

alpha = 0.05

In [6]:
dict_df_neighbors = get_k_neighbors(df=df, cf_df=cf_df, 
                                    k=n, 
                                    feat_trgt=feat_trgt, feat_trgt_vals=feat_trgt_vals, 
                                    feat_rlvt=feat_rlvt, 
                                    feat_prot=feat_prot, feat_prot_vals=feat_prot_vals, 
                                    standardize=True, 
                                   )

target feature LoanApproval with values {'pos': 1, 'neg': -1}
protected feature Gender with values {'non_protected': 0, 'protected': 1}
with relevant features ['AnnualSalary', 'AccountBalance']
all features: ['LoanApproval', 'AnnualSalary', 'AccountBalance', 'Gender']
standardizing


In [7]:
test_disc = get_wald_ci(dict_df_neighbors=dict_df_neighbors,
                       feat_trgt=feat_trgt, feat_trgt_vals=feat_trgt_vals,
                       alpha=alpha,
                       )

In [9]:
test_disc[test_disc['cfST'] == 'Yes']

,individual,p1,p2,org_diff,d_alpha,diff,CIs,cfST
4,9,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes
14,44,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes
15,47,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes
17,50,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes
19,55,0.818182,0.0,0.818,0.227931,0.590251,"[0.5902508705143656, 1.0461127658492708]",Yes
...,...,...,...,...,...,...,...,...
1691,4918,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes
1697,4946,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes
1700,4958,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes
1703,4973,1.000000,0.0,1.000,0.000000,1.000000,"[1.0, 1.0]",Yes


In [ ]:
# 4) We need to start testing each group: do now for ind


ind = 0
int(ind)

In [ ]:

ctr_group = dict_df_neighbors[ind]['control']
# ctr_group = ctr_group.merge(df[feat_list], how='inner', left_on='org_index', right_index=True)

ctr_group

In [ ]:
# for ind ... need to merge with df and cf_df for rest of features:
tst_group = dict_df_neighbors[ind]['test']
# ctr_group = ctr_group.merge(df[feat_list], how='inner', left_on='org_index', right_index=True)

tst_group

In [ ]:
k1=ctr_group.shape[0]
print(k1)
k2=tst_group.shape[0]
print(k2)

In [ ]:
ctr_group[ctr_group[feat_trgt]==feat_trgt_vals['neg']].shape[0]

In [ ]:
p1 = ctr_group[ctr_group[feat_trgt]==feat_trgt_vals['neg']].shape[0] / ctr_group.shape[0]
print(p1)
p2 = tst_group[tst_group[feat_trgt]==feat_trgt_vals['neg']].shape[0] / tst_group.shape[0]
print(p2)

diff = p1 - p2
diff

In [ ]:
def get_wald_ci(alpha: float, p1: float, p2: float, k1: int, k2: int):
    
    wald_ci_summary = {}
    
    z_score = round(st.norm.ppf(1 - (alpha/2)), 2)
    d_alpha = z_score * math.sqrt( (p1*(1 - p1)/k1) + (p2*(1 - p2)/k2) )
    
    wald_ci_summary['d_alpha'] = d_alpha
    wald_ci_summary['CIs'] = [(p1 - p2) - d_alpha, (p1 - p2) + d_alpha]
    if (p1 - p2) >= 0:
        wald_ci_summary['diff'] = max(0, p1 - p2 - d_alpha)
    else:
        wald_ci_summary['diff'] = min(0, p1 - p2 + d_alpha)
    
    print(wald_ci_summary)
    
    return wald_ci_summary

In [ ]:
ind_wald_ci = get_wald_ci(alpha=alpha, p1=p1, p2=p2, k1=ctr_group.shape[0], k2=tst_group.shape[0])

In [ ]:
# todo: what's the output for the ST: yes/no where?

In [ ]:
d = []
for p in game.players.passing():
    d.append(
        {
            'Player': p,
            'Team': p.team,
            'Passer Rating':  p.passer_rating()
        }
    )

pd.DataFrame(d)

In [ ]:
print(ind)
print(ind_wald_ci)

In [ ]:
# i guess i need some sort of threshold here???
diff_epsilon = 0.05 # tau in the second paper

In [ ]:
if ind_wald_ci['diff'] > diff_epsilon:
    ind_wald_ci['cfST'] = 'Yes'
else:
    ind_wald_ci['cfST'] = 'No'
    
#how to return the results? get df with discrimination columns!

In [ ]:
from sklearn.neighbors import BallTree
rng = np.random.RandomState(0)
X = rng.random_sample((10, 3))
print(X)
print('---')
print(X[:1])
tree = BallTree(X, leaf_size=2)              
dist, ind = tree.query(X[:1], k=3)                
print(ind)  # indices of 3 closest neighbors
#[0 3 1]
print(dist)  # distances to 3 closest neighbors
#[ 0.          0.19662693  0.29473397]